##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Crie um pipeline TFX para seus dados com o template Penguin

---


Observação: recomendamos executar este tutorial no Google Cloud [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench). [Acessar o Vertex AI Workbench](https://console.cloud.google.com/vertex-ai/workbench).

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_template"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_template.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/penguin_template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

## Introdução

Este documento fornecerá instruções para criar um pipeline TensorFlow Extended (TFX) para seu próprio dataset usando o *template Penguin* fornecido com o pacote TFX em Python. O pipeline criado usará inicialmente o dataset [Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html), mas vamos transformar o pipeline para seu dataset.


### Pré-requisitos

- Linux / MacOS
- Python 3.6-3.8
- Jupyter notebook


## Etapa 1. Copie o template predefinido para o diretório do seu projeto.

Nesta etapa, criaremos um diretório de projeto de pipeline funcional e arquivos copiando os arquivos do *template Penguin* no TFX. Você pode pensar nisso como uma estrutura básica para seu projeto de pipeline TFX.

### Atualize o pip

Se estivermos rodando no Colab, devemos ter certeza de que temos a versão mais recente do Pip. É claro que os sistemas locais podem ser atualizados separadamente.

Observação: a atualização provavelmente também é uma boa ideia se você estiver trabalhando no Vertex AI Workbench.

In [ ]:
import sys
if 'google.colab' in sys.modules:
  !pip install --upgrade pip

### Instale o pacote obrigatório

Primeiro, instale o TFX e o TensorFlow Model Analysis (TFMA).


In [ ]:
!pip install -U tfx tensorflow-model-analysis

Vamos verificar as versões do TFX.

In [ ]:
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tfx

print('TF version: {}'.format(tf.__version__))
print('TFMA version: {}'.format(tfma.__version__))
print('TFX version: {}'.format(tfx.__version__))

Estamos prontos para criar um pipeline.

Defina o `PROJECT_DIR` para o destino apropriado para seu ambiente. O valor padrão é `~/imported/${PIPELINE_NAME}`, que é apropriado para o ambiente do [Google Cloud AI Platform Notebook](https://console.cloud.google.com/ai-platform/notebooks/).

Você pode dar um nome diferente ao seu pipeline alterando `PIPELINE_NAME` abaixo. Este também se tornará o nome do diretório do projeto onde seus arquivos serão colocados.


In [ ]:
PIPELINE_NAME="my_pipeline"
import os
# Set this project directory to your new tfx pipeline project.
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "imported", PIPELINE_NAME)

### Copie os arquivos do template.

O TFX inclui o template `penguin` com o pacote TFX python. O template `penguin` contém muitas instruções para trazer seu dataset para o pipeline, que é o objetivo deste tutorial.

O comando CLI `tfx template copy` copia arquivos de template predefinidos no diretório do projeto.

In [ ]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=penguin

Altere o contexto do diretório de trabalho neste notebook para o diretório do projeto.

In [ ]:
%cd {PROJECT_DIR}

> OBSERVAÇÃO: Se você estiver usando o JupyterLab ou o Google Cloud AI Platform Notebook, não se esqueça de alterar o diretório no `File Browser` à esquerda clicando no diretório do projeto assim que ele for criado.

### Inspecione os arquivos-fonte copiados

O template TFX fornece arquivos de estrutura básicos para construir um pipeline, incluindo código-fonte Python e dados de amostra. O template `penguin` usa o mesmo dataset e modelo de ML *Palmer Penguins* que o [exemplo Penguin](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Aqui está uma breve introdução a cada um dos arquivos Python.

- `pipeline` - este diretório contém a definição do pipeline
    - `configs.py` — define constantes comuns para executores de pipeline
    - `pipeline.py` — define componentes TFX e um pipeline
- `models` - este diretório contém definições de modelo de ML.
    - `features.py`, `features_test.py` — define características para o modelo
    - `preprocessing.py`, `preprocessing_test.py` — define rotinas de pré-processamento para os dados
    - `constants.py` — define constantes do modelo
    - `model.py` , `model_test.py` – define o modelo de ML usando frameworks de ML como o TensorFlow
- `local_runner.py` — define um executor para ambiente local que usa mecanismo de orquestração local
- `kubeflow_runner.py` — define um executor para o mecanismo de orquestração do Kubeflow Pipelines


Por padrão, o template inclui apenas componentes TFX padrão. Se precisar de algumas ações personalizadas, você pode criar componentes personalizados para seu pipeline. Veja o [guia de componentes personalizados do TFX](https://www.tensorflow.org/tfx/guide/understanding_custom_components) para mais detalhes.

#### Arquivos de teste de unidade.

Você poderá perceber que existem alguns arquivos com `_test.py` em seus nomes. Estes são testes unitários do pipeline e é recomendado adicionar mais testes unitários à medida que você for implementando seus próprios pipelines. Você pode executar testes de unidade fornecendo o nome do módulo dos arquivos de teste com o sinalizador `-m`. Geralmente você pode obter um nome de módulo excluindo a extensão `.py` e substituindo `/` por `.`. Por exemplo:

In [ ]:
import sys
!{sys.executable} -m models.features_test

### Crie um pipeline TFX em ambiente local.

O TFX oferece suporte a vários mecanismos de orquestração para executar pipelines. Usaremos um mecanismo de orquestração local. O mecanismo de orquestração local é executado sem quaisquer dependências adicionais e é adequado para desenvolvimento e depuração porque é executado em ambiente local, em vez de depender de clusters de computação remotos.

Usaremos o `local_runner.py` para executar seu pipeline usando o orquestrador local. Você precisa criar um pipeline antes de executá-lo. Você pode criar um pipeline com o comando `pipeline create`.


In [ ]:
!tfx pipeline create --engine=local --pipeline_path=local_runner.py

O comando `pipeline create` registra seu pipeline definido em `local_runner.py` sem realmente executá-lo.

Você executará o pipeline criado com o comando `run create` nas etapas a seguir.


## Etapa 2. Ingira os SEUS dados no pipeline

O pipeline inicial ingere o dataset do pinguim incluído no template. Você precisa colocar seus dados no pipeline, e a maioria dos pipelines do TFX começa com o componente ExampleGen.

### Escolha um ExampleGen

Seus dados podem ser armazenados em qualquer lugar que seu pipeline possa acessar, num sistema de arquivos local ou distribuído, ou num sistema com capacidade de pesquisa. O TFX fornece vários [componentes `ExampleGen`](https://www.tensorflow.org/tfx/guide/examplegen) para trazer seus dados para um pipeline do TFX. Você pode escolher um dos seguintes componentes geradores de exemplo.

- CsvExampleGen: lê arquivos CSV em um diretório. Usado no [exemplo Penguin](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin) e no [exemplo Chicago Taxi](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline).
- ImportExampleGen: recebe arquivos TFRecord com formato de dados TF Example. Usado em [exemplos com MNIST](https://github.com/tensorflow/tfx/tree/master/tfx/examples/mnist).
- FileBasedExampleGen para formato [Avro](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/avro_executor.py) ou [Parquet](https://github.com/tensorflow/tfx/blob/master/tfx/components/example_gen/custom_executors/parquet_executor.py).
- [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen): lê dados diretamente no Google Cloud BigQuery. Usado em [exemplos do Chicago Taxi](https://github.com/tensorflow/tfx/tree/master/tfx/examples/chicago_taxi_pipeline).

Você também pode criar seu próprio ExampleGen, por exemplo, tfx inclui [um ExampleGen personalizado que usa o Presto](https://github.com/tensorflow/tfx/tree/master/tfx/examples/custom_components/presto_example_gen) como fonte de dados. Consulte o [guia](https://www.tensorflow.org/tfx/guide/examplegen#custom_examplegen) para obter mais informações sobre como usar e desenvolver executores customizados.

Depois de decidir qual ExampleGen usar, você precisará modificar a definição do pipeline para usar seus dados.

1. Altere o `DATA_PATH` em `local_runner.py` e defina-o como o local dos seus arquivos.

- Se você tiver arquivos em ambiente local, especifique o caminho. Esta é a melhor opção para desenvolver ou depurar um pipeline.
- Se os arquivos estiverem armazenados no GCS, você poderá usar um caminho começando com `gs://{bucket_name}/...`. Certifique-se de que você pode acessar o GCS a partir do seu terminal, por exemplo, usando [`gsutil`](https://cloud.google.com/storage/docs/gsutil). Siga [o guia de autorização no Google Cloud](https://cloud.google.com/sdk/docs/authorizing) se necessário.
- Se quiser usar um ExampleGen baseado em consulta como BigQueryExampleGen, você precisará de uma instrução Query para selecionar dados da fonte de dados. Há mais algumas coisas que você precisa definir para usar o Google Cloud BigQuery como fonte de dados.
    - Em `pipeline/configs.py`:
        - Altere `GOOGLE_CLOUD_PROJECT` e `GCS_BUCKET_NAME` para seu projeto GCP e nome do bucket. O bucket deve existir antes de executarmos o pipeline.
        - Descomente a variável `BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS`.
        - Descomente e defina a variável `BIG_QUERY_QUERY` para **sua instrução de query**.
    - Em `local_runner.py`:
        - Comente o argumento `data_path` e descomente o argumento `query` em `pipeline.create_pipeline()`.
    - Em `pipeline/pipeline.py`:
        - Comente o argumento `data_path` e descomente o argumento `query` em `create_pipeline()`.
        - Use [BigQueryExampleGen](https://www.tensorflow.org/tfx/api_docs/python/tfx/extensions/google_cloud_big_query/example_gen/component/BigQueryExampleGen) em vez de CsvExampleGen.

1. Substitua o CsvExampleGen existente pela sua classe ExampleGen em `pipeline/pipeline.py`. Cada classe ExampleGen possui uma assinatura diferente. Consulte o [guia do componente ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) para mais detalhes. Não esqueça de importar os módulos necessários com instruções `import` em `pipeline/pipeline.py`.

O pipeline inicial consiste em quatro componentes, `ExampleGen`, `StatisticsGen`, `SchemaGen` e `ExampleValidator`. Não precisamos alterar nada para `StatisticsGen`, `SchemaGen` e `ExampleValidator`. Vamos executar o pipeline pela primeira vez.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Você deverá ver a mensagem "Component ExampleValidator is finished." se o pipeline foi executado com sucesso.

### Examine a saída do pipeline.

O pipeline TFX produz dois tipos de saída, artefatos e um [banco de dados de metadados (MLMD)](https://www.tensorflow.org/tfx/guide/mlmd) que contém metadados de artefatos e execuções de pipeline. O local da saída é definido em `local_runner.py`. Por padrão, os artefatos são armazenados no diretório `tfx_pipeline_output` e os metadados são armazenados como um banco de dados sqlite no diretório `tfx_metadata`.

Você pode usar APIs MLMD para examinar essas saídas. Primeiro, definiremos algumas funções utilitárias para pesquisar artefatos de saída que acabaram de ser produzidos.

In [ ]:
import tensorflow as tf
import tfx
from ml_metadata import errors
from ml_metadata.proto import metadata_store_pb2
from tfx.types import artifact_utils

# TODO(b/171447278): Move these functions into TFX library.

def get_latest_executions(store, pipeline_name, component_id = None):
  """Fetch all pipeline runs."""
  if component_id is None:  # Find entire pipeline runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('run')
        if c.properties['pipeline_name'].string_value == pipeline_name
    ]
  else:  # Find specific component runs.
    run_contexts = [
        c for c in store.get_contexts_by_type('component_run')
        if c.properties['pipeline_name'].string_value == pipeline_name and
           c.properties['component_id'].string_value == component_id
    ]
  if not run_contexts:
    return []
  # Pick the latest run context.
  latest_context = max(run_contexts,
                       key=lambda c: c.last_update_time_since_epoch)
  return store.get_executions_by_context(latest_context.id)

def get_latest_artifacts(store, pipeline_name, component_id = None):
  """Fetch all artifacts from latest pipeline execution."""
  executions = get_latest_executions(store, pipeline_name, component_id)

  # Fetch all artifacts produced from the given executions.
  execution_ids = [e.id for e in executions]
  events = store.get_events_by_execution_ids(execution_ids)
  artifact_ids = [
      event.artifact_id for event in events
      if event.type == metadata_store_pb2.Event.OUTPUT
  ]
  return store.get_artifacts_by_id(artifact_ids)

def find_latest_artifacts_by_type(store, artifacts, artifact_type):
  """Get the latest artifacts of a specified type."""
  # Get type information from MLMD
  try:
    artifact_type = store.get_artifact_type(artifact_type)
  except errors.NotFoundError:
    return []
  # Filter artifacts with type.
  filtered_artifacts = [aritfact for aritfact in artifacts
                        if aritfact.type_id == artifact_type.id]
  # Convert MLMD artifact data into TFX Artifact instances.
  return [artifact_utils.deserialize_artifact(artifact_type, artifact)
      for artifact in filtered_artifacts]


from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

import pprint

from tfx.orchestration import metadata
from tfx.types import artifact_utils
from tfx.types import standard_artifacts

def preview_examples(artifacts):
  """Preview a few records from Examples artifacts."""
  pp = pprint.PrettyPrinter()
  for artifact in artifacts:
    print("==== Examples artifact:{}({})".format(artifact.name, artifact.uri))
    for split in artifact_utils.decode_split_names(artifact.split_names):
      print("==== Reading from split:{}".format(split))
      split_uri = artifact_utils.get_split_uri([artifact], split)

      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = [os.path.join(split_uri, name)
                            for name in os.listdir(split_uri)]
      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames,
                                        compression_type="GZIP")
      # Iterate over the first 2 records and decode them.
      for tfrecord in dataset.take(2):
        serialized_example = tfrecord.numpy()
        example = tf.train.Example()
        example.ParseFromString(serialized_example)
        pp.pprint(example)

import local_runner

metadata_connection_config = metadata.sqlite_metadata_connection_config(
              local_runner.METADATA_PATH)

Agora podemos ler metadados de artefatos de saída do MLMD.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous pipeline run.
    artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
    # Find artifacts of Examples type.
    examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)
    # Find artifacts generated from StatisticsGen.
    stats_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleStatistics.TYPE_NAME)
    # Find artifacts generated from SchemaGen.
    schema_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Schema.TYPE_NAME)
    # Find artifacts generated from ExampleValidator.
    anomalies_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.ExampleAnomalies.TYPE_NAME)

Agora podemos examinar os resultados de cada componente. [Tensorflow Data Validation (TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started) é usado em `StatisticsGen`, `SchemaGen` e `ExampleValidator`, e TFDV pode ser usado para visualizar os resultados desses componentes.

Neste tutorial, usaremos métodos helper de visualização no TFX que usam TFDV internamente para mostrar a visualização. Consulte o [tutorial dos componentes TFX](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) para saber mais sobre cada componente.

#### Examine a saída do ExampleGen

Vamos examinar a saída de ExampleGen. Dê uma olhada nos dois primeiros exemplos para cada divisão:

In [ ]:
preview_examples(examples_artifacts)

Por padrão, o TFX ExampleGen divide os exemplos em duas divisões, *train* e *eval*, mas você pode [ajustar a configuração da divisão](https://www.tensorflow.org/tfx/guide/examplegen#span_version_and_split).

#### Examine a saída do StatisticsGen


In [ ]:
visualize_artifacts(stats_artifacts)

Essas estatísticas são fornecidas ao SchemaGen para construir um esquema de dados automaticamente.

#### Examine a saída do SchemaGen


In [ ]:
visualize_artifacts(schema_artifacts)

Este esquema é inferido automaticamente da saída do StatisticsGen. Usaremos esse esquema gerado neste tutorial, mas você também pode [modificar e personalizar o esquema](https://www.tensorflow.org/tfx/guide/statsgen#creating_a_curated_schema).

#### Examine a saída de ExampleValidator


In [ ]:
visualize_artifacts(anomalies_artifacts)

Se alguma anomalia foi encontrada, você poderá revisar seus dados para que todos os exemplos sigam suas suposições. Os resultados de outros componentes como StatisticsGen podem ser úteis. As anomalias encontradas não bloqueiam a execução do pipeline.

Você poderá ver as características disponíveis nas saídas do `SchemaGen`. Se suas características puderem ser usadas ​​para construir o modelo de ML diretamente no `Trainer`, você pode pular a próxima etapa e ir para a Etapa 4. Caso contrário, você terá que fazer uma engenharia de recursos na próxima etapa. O componente `Transform` é necessário quando são necessárias operações de full-pass, como cálculo de médias, especialmente quando você precisa escalar.

## Etapa 3. (Opcional) Engenharia de características com o componente Transform.

Nesta etapa, você definirá vários jobs de engenharia de características que serão usados ​​pelo componente `Transform` no pipeline. Consulte o [guia do componente Transform](https://www.tensorflow.org/tfx/guide/transform) para mais informações.

Isso só será necessário se o código de treinamento exigir características adicionais que não estão disponíveis na saída de ExampleGen. Caso contrário, sinta-se à vontade para avançar para a próxima etapa de uso do Trainer.

### Defina as características do modelo

`models/features.py` contém constantes para definir características para o modelo, incluindo nomes de características, tamanho do vocabulário e assim por diante. Por padrão, o modelo `penguin` tem duas constantes, `FEATURE_KEYS` e `LABEL_KEY`, porque nosso modelo `penguin` resolve um problema de classificação usando aprendizado supervisionado e todas as características são características numéricas contínuas. Veja as [definições de características do exemplo Chicago Taxi](https://github.com/tensorflow/tfx/blob/master/tfx/experimental/templates/taxi/models/features.py) como outro exemplo.


### Implemente pré-processamento para treinamento/serviço em preprocessing_fn().

A engenharia de características real acontece na função `preprocessing_fn()` em `models/preprocessing.py`.

Em `preprocessing_fn` você pode definir uma série de funções que manipulam o dict de tensores de entrada para produzir o dict de tensores de saída. Tem as funções helper como `scale_to_0_1` e `compute_and_apply_vocabulary` na API TensorFlow Transform ou você pode simplesmente usar funções regulares do TensorFlow. Por padrão, o template `penguin` inclui exemplos de uso da função [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score) para normalizar os valores das características.

Consulte o guia [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) para mais informações sobre a criação de `preprocessing_fn`.


### Inclua o componente Transform no pipeline.

Se seu preprocessing_fn estiver pronto, adicione o componente `Transform` ao pipeline.

1. No arquivo `pipeline/pipeline.py`, descomente `# components.append(transform)` para adicionar o componente ao pipeline.

Você pode atualizar o pipeline e executá-lo novamente.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Se o pipeline foi executado com sucesso, você verá "Component Transform is finished" *em algum lugar* do log. Como o componente `Transform` e o componente `ExampleValidator` não dependem um do outro, a ordem das execuções não é fixa. Dito isto, `Transform` e `ExampleValidator` podem ser o último componente na execução do pipeline.

### Examine a saída do Transform

O componente Transform cria dois tipos de saída, um grafo TensorFlow e exemplos transformados. Os exemplos transformados são do tipo de artefato Examples que também é produzido por ExampleGen, mas este contém valores de características transformadas.

Você pode examiná-las como fizemos na etapa anterior.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
    # Search all aritfacts from the previous run of Transform component.
    artifacts = get_latest_artifacts(metadata_handler.store,
                                     PIPELINE_NAME, "Transform")
    # Find artifacts of Examples type.
    transformed_examples_artifacts = find_latest_artifacts_by_type(
        metadata_handler.store, artifacts,
        standard_artifacts.Examples.TYPE_NAME)

In [ ]:
preview_examples(transformed_examples_artifacts)

## Etapa 4. Treine seu modelo com o componente Trainer.

Construiremos um modelo de ML usando o componente `Trainer`. Consulte o [guia do componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) para obter mais informações. Você precisa fornecer o código do seu modelo para o componente Trainer.

### Defina seu modelo.

No template Penguin, `models.model.run_fn` é usado como argumento `run_fn` para o componente `Trainer`. Isto significa que a função `run_fn()` em `models/model.py` será chamada quando o componente `Trainer` for executado. Você pode ver o código para construir um modelo DNN simples usando a API `keras` em determinado código. Consulte o guia [TensorFlow 2.x no TFX](https://www.tensorflow.org/tfx/guide/keras) para mais informações sobre como usar a API keras no TFX.

Neste `run_fn`, você deve construir um modelo e salvá-lo em um diretório apontado por `fn_args.serving_model_dir` que é especificado pelo componente. Você pode usar outros argumentos em `fn_args` que são passados ​​para `run_fn`. Consulte os [códigos relacionados](https://github.com/tensorflow/tfx/blob/b01482442891a49a1487c67047e85ab971717b75/tfx/components/trainer/executor.py#L141) para obter a lista completa de argumentos em `fn_args`.

Defina suas características em `models/features.py` e use-as conforme necessário. Se você transformou suas características na Etapa 3, deverá usar as características transformadas como entradas para seu modelo.

### Adicione o componente Trainer ao pipeline.

Se o seu run_fn estiver pronto, adicione o componente `Trainer` ao pipeline.

1. No arquivo `pipeline/pipeline.py`, descomente `# components.append(trainer)` para adicionar o componente ao pipeline.

Os argumentos para o componente trainer podem depender de você usar o componente Transform ou não.

- Se você **NÃO** usa o componente `Transform`, não precisa alterar os argumentos.

- Se você usar o componente `Transform`, precisará alterar os argumentos ao criar uma instância do componente `Trainer`.

    - Altere o argumento de `examples` para `examples=transform.outputs['transformed_examples'],`. Precisamos usar exemplos transformados para treinamento.
    - Adicione o argumento `transform_graph` como `transform_graph=transform.outputs['transform_graph'],`. Este grafo contém o grafo do TensorFlow para as operações de transformação.
    - Após as alterações acima, o código para criação do componente Trainer ficará semelhante ao seguinte.

    ```python
    # If you use a Transform component.
    trainer = Trainer(
        run_fn=run_fn,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        schema=schema_gen.outputs['schema'],
        ...
    ```

Você pode atualizar o pipeline e executá-lo novamente.

In [ ]:
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Quando esta execução for executada com sucesso, você terá criado e executado seu primeiro pipeline TFX para seu modelo. Parabéns!

Seu novo modelo estará localizado em algum lugar no diretório de saída, mas seria melhor ter um modelo em local fixo ou serviço fora do pipeline do TFX que contenha muitos resultados provisórios. Melhor ainda com avaliação contínua do modelo construído, que é fundamental em sistemas de ML em produção. Veremos como a avaliação contínua e as implantações funcionam no TFX na próxima etapa.

## Etapa 5. (Opcional) Avalie o modelo com o Evaluator e publique com o pusher.


O componente [`Evaluator`](https://www.tensorflow.org/tfx/guide/evaluator) avalia continuamente cada modelo construído no `Trainer` e [`Pusher`](https://www.tensorflow.org/tfx/guide/pusher) copia o modelo para um local predefinido no sistema de arquivos ou até mesmo para o [Google Cloud AI Platform Models](https://console.cloud.google.com/ai-platform/models).

### Adiciona o componente Evaluator ao pipeline.

No arquivo `pipeline/pipeline.py`:

1. Descomente `# components.append(model_resolver)` para adicionar o resolvedor de modelo mais recente ao pipeline. O Evaluator pode ser usado para comparar um modelo com o modelo de referência antigo que passou no Evaluator na última execução do pipeline. `LatestBlessedModelResolver` encontra o modelo mais recente que passou no Evaluator.
2. Defina um `tfma.MetricsSpec` adequado para seu modelo. A avaliação pode ser diferente para cada modelo de ML. No template Penguin, `SparseCategoricalAccuracy` foi usado porque estamos resolvendo um problema de classificação multicategoria. Você também precisa especificar `tfma.SliceSpec` para analisar seu modelo para fatias específicas. Para mais detalhes, consulte o [Guia do componente Evaluator](https://www.tensorflow.org/tfx/guide/evaluator).
3. Descomente `# components.append(evaluator)` para adicionar o componente ao pipeline.

Você pode atualizar o pipeline e executá-lo novamente.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

### Examine a saída do Evaluator

Esta etapa requer a extensão do notebook Jupyter do TensorFlow Model Analysis (TFMA). Observe que a versão da extensão do notebook TFMA deve ser idêntica à versão do pacote python TFMA.

O comando a seguir instalará a extensão do notebook TFMA do registro NPM. Pode levar vários minutos para ser concluído.

In [ ]:
# Install TFMA notebook extension.
!jupyter labextension install tensorflow_model_analysis@{tfma.__version__}

Se a instalação estiver concluída, **reinicie seu navegador** para que a extensão tenha efeito.

In [ ]:
with metadata.Metadata(metadata_connection_config) as metadata_handler:
  # Search all aritfacts from the previous pipeline run.
  artifacts = get_latest_artifacts(metadata_handler.store, PIPELINE_NAME)
  model_evaluation_artifacts = find_latest_artifacts_by_type(
      metadata_handler.store, artifacts,
      standard_artifacts.ModelEvaluation.TYPE_NAME)

In [ ]:
if model_evaluation_artifacts:
  tfma_result = tfma.load_eval_result(model_evaluation_artifacts[0].uri)
  tfma.view.render_slicing_metrics(tfma_result)

### Adiciona o componente Pusher ao pipeline.

Se o modelo parecer promissor, precisamos publicá-lo. [O componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) pode publicar o modelo em um local do sistema de arquivos ou para GCP AI Platform Models usando [um executor personalizado](https://github.com/tensorflow/tfx/blob/master/tfx/extensions/google_cloud_ai_platform/pusher/executor.py).

O componente <a><code data-md-type="codespan">Evaluator</code></a> avalia continuamente cada modelo construído no <code>Trainer</code> e <a><code>Pusher</code></a> copia o modelo para um local predefinido no sistema de arquivos ou até mesmo para o <a>Google Cloud AI Platform Models</a>.

1. Em `local_runner.py`, defina `SERVING_MODEL_DIR` como um diretório para publicar.
2. No arquivo `pipeline/pipeline.py`, descomente `# components.append(pusher)` para adicionar o Pusher ao pipeline.

Você pode atualizar o pipeline e executá-lo novamente.

In [ ]:
# Update and run the pipeline.
!tfx pipeline update --engine=local --pipeline_path=local_runner.py \
 && tfx run create --engine=local --pipeline_name={PIPELINE_NAME}

Você deverá encontrar seu novo modelo em `SERVING_MODEL_DIR`.

## Etapa 6. (Opcional) Implante seu pipeline para o Kubeflow Pipelines no GCP.


Conforme mencionado anteriormente, `local_runner.py` é bom para fins de depuração ou desenvolvimento, mas não é a melhor solução para cargas de trabalho em produção. Nesta etapa, implantaremos o pipeline no Kubeflow Pipelines no Google Cloud.

### Preparação

Precisamos do pacote `kfp` python e do programa `skaffold` para implantar um pipeline em um cluster Kubeflow Pipelines.

In [ ]:
!pip install --upgrade -q kfp

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold

Você precisa mover o binário `skaffold` para o local onde seu shell possa encontrá-lo. Ou você pode especificar o caminho para o skaffold ao executar o binário `tfx` com o sinalizador `--skaffold-cmd` .

In [ ]:
# Move skaffold binary into your path
!mv skaffold /home/jupyter/.local/bin/

Você também precisa de um cluster Kubeflow Pipelines para executar o pipeline. Siga as etapas 1 e 2 no [Tutorial TFX on Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

Quando seu cluster estiver pronto, abra o painel do pipeline clicando em *Open Pipelines Dashboard* na [página `Pipelines` do Google Cloud Console](http://console.cloud.google.com/ai-platform/pipelines). A URL desta página é `ENDPOINT` para solicitar a execução de um pipeline. O valor do endpoint é tudo na URL depois di https://, até e incluindo googleusercontent.com. Coloque seu endpoint no seguinte bloco de código.


In [ ]:
ENDPOINT='' # Enter your ENDPOINT here.

Para executar nosso código num cluster Kubeflow Pipelines, precisamos compactar nosso código numa imagem de container. A imagem será construída automaticamente durante a implantação do nosso pipeline, e você só precisa definir um nome e um registro de container para sua imagem. Em nosso exemplo, usaremos o [Google Container Registry](https://cloud.google.com/container-registry) e o nomearemos `tfx-pipeline`.

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]

# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

### Defina a localização dos dados.

Seus dados devem estar acessíveis no cluster Kubeflow Pipelines. Se você usou dados em seu ambiente local, talvez seja necessário carregá-los para um armazenamento remoto, como o Google Cloud Storage. Por exemplo, podemos fazer upload de dados do Penguin para um bucket padrão que é criado automaticamente quando um cluster Kubeflow Pipelines é implantado como a seguir.

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/penguin/

Atualize o local dos dados armazenados em `DATA_PATH` em `kubeflow_runner.py`.

Se você estiver usando BigQueryExampleGen, não há necessidade de fazer upload do arquivo de dados, mas certifique-se de que `kubeflow_runner.py` use a mesma `query` e argumento `beam_pipeline_args` para a função `pipeline.create_pipeline()`.

### Implante o pipeline.

Se tudo estiver pronto, você pode criar um pipeline usando o comando `tfx pipeline create`.

> Observação: ao criar um pipeline para o Kubeflow Pipelines, precisamos de uma imagem de container que será usada para executar nosso pipeline. E o `skaffold` vai criar a imagem para nós. Como o `skaffold` extrai imagens base do docker hub, levará de 5 a 10 minutos quando criarmos a imagem pela primeira vez, mas levará muito menos tempo na segunda vez.


In [ ]:
!tfx pipeline create  \
--engine=kubeflow \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

Agora inicie uma execução com o pipeline recém-criado usando o comando `tfx run create`.

In [ ]:
!tfx run create --engine=kubeflow --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

Ou você também pode executar o pipeline no painel do Kubeflow Pipelines. A nova execução será listada em `Experiments` no Painel do Kubeflow Pipelines. Clicar no experimento permitirá monitorar o progresso e visualizar os artefatos criados durante a execução.

Se você estiver interessado em executar seu pipeline no Kubeflow Pipelines, procure mais instruções no [Tutorial TFX on Cloud AI Platform Pipelines](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines).

### Limpeza

Para limpar todos os recursos do Google Cloud usados ​​nesta etapa, [exclua o projeto do Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) usado no tutorial.

Como alternativa, você pode limpar recursos individuais acessando cada console individualmente:

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)